In [1]:
import sys

sys.path.append("../src")
import numpy as np
import pandas as pd

from ranking_sim import (
    process_raw_data,
    rank_by_brier,
    rank_by_bss,
)

In [2]:
INPUT_FOLDER = "../data/raw"

In [3]:
df = process_raw_data(f"{INPUT_FOLDER}/leaderboard_human.pkl")
df.head()

,id,source,direction,forecast_due_date,market_value_on_due_date,resolution_date,resolved_to,resolved,forecast,imputed,score,horizon,model,organization,question_id,question_type
0,1285,infer,(),2024-07-21,0.0163,2024-07-28,0.0,True,0.50,False,0.2500,7,Claude-2.1 (zero shot),Anthropic,infer-1285-7-2024-07-21,market
2,1342,infer,(),2024-07-21,0.0088,2024-07-28,0.0,True,0.50,False,0.2500,7,Claude-2.1 (zero shot),Anthropic,infer-1342-7-2024-07-21,market
3,1353,infer,(),2024-07-21,0.4943,2024-07-28,0.0,True,0.50,False,0.2500,7,Claude-2.1 (zero shot),Anthropic,infer-1353-7-2024-07-21,market
5,1373,infer,(),2024-07-21,0.7474,2024-07-28,0.0,True,0.60,False,0.3600,7,Claude-2.1 (zero shot),Anthropic,infer-1373-7-2024-07-21,market
6,1374,infer,(),2024-07-21,0.7568,2024-07-28,1.0,True,0.51,False,0.2401,7,Claude-2.1 (zero shot),Anthropic,infer-1374-7-2024-07-21,market


In [4]:
mask = df['question_type'] == 'market'
df.loc[mask, 'horizon'].unique()

array([7])

In [5]:
mask = df['question_type'] == 'market'
df.loc[mask, ['source', 'id', 'model', 'forecast']].duplicated().any()

np.False_